# Ridge και Lasso Regression με το dataset House Prices (Kaggle)


## Θεωρία: Ridge & Lasso Regularization

Για να μειώσουμε την αστάθεια και το overfitting, προσθέτουμε όρους regularization στη συνάρτηση κόστους. Γενικά:
$$
\text{Loss} = \frac{1}{n}\lVert \mathbf{y} - X\boldsymbol{\beta} \rVert_2^2 + \alpha\,R(\boldsymbol{\beta})
$$
Όπου $\alpha>0$ ελέγχει τη αυστηρότητα της ποινής.

- Ridge (L2): $R(\boldsymbol{\beta})=\lVert\boldsymbol{\beta}\rVert_2^2 = \sum_j \beta_j^2$
  $$\min_{\boldsymbol{\beta}}\; \frac{1}{n}\lVert \mathbf{y} - X\boldsymbol{\beta} \rVert_2^2 + \alpha \sum_j \beta_j^2$$
  Κλειστή λύση: $\boldsymbol{\beta}_{\text{Ridge}}=(X^\top X + \alpha I)^{-1} X^\top \mathbf{y}$
- Lasso (L1): $R(\boldsymbol{\beta})=\lVert\boldsymbol{\beta}\rVert_1 = \sum_j |\beta_j|$
  $$\min_{\boldsymbol{\beta}}\; \frac{1}{n}\lVert \mathbf{y} - X\boldsymbol{\beta} \rVert_2^2 + \alpha \sum_j |\beta_j|$$
  Προάγει αραιότητα (αρκετοί συντελεστές γίνονται 0).

Ερμηνεία του $\alpha$:
- Μικρό $\alpha$ → κοντά στην OLS → πιθανό overfitting.
- Μεγάλο $\alpha$ → έντονο shrinkage → underfitting.
- Ενδιάμεσο $\alpha$ → ισορροπία bias/variance και καλύτερη γενίκευση.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


## 1. Φόρτωση και προετοιμασία δεδομένων

In [ ]:
ROOT = Path('..')
DATA_PATH = ROOT / 'data' / 'house_prices_train.csv'

df = pd.read_csv(DATA_PATH)

feature_names = [
    'LotArea',
    'OverallQual',
    'OverallCond',
    'YearBuilt',
    'GrLivArea',
    'BedroomAbvGr',
    'GarageCars',
]
target_name = 'SalePrice'

df_model = df[feature_names + [target_name]].copy()
df_model = df_model.dropna(subset=[target_name])
df_model[feature_names] = df_model[feature_names].fillna(
    df_model[feature_names].median()
)

X = df_model[feature_names]
y = df_model[target_name]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

X_train.shape, X_test.shape


## 2. Σύγκριση Linear, Ridge, Lasso για συγκεκριμένα `alpha`

In [ ]:
def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2 = r2_score(y_test, y_pred_test)
    return {'model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2}


models = {
    'Linear': Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())]),
    'Ridge (α=1.0)': Pipeline([('scaler', StandardScaler()), ('model', Ridge(alpha=1.0, random_state=0))]),
    'Lasso (α=0.1)': Pipeline([('scaler', StandardScaler()), ('model', Lasso(alpha=0.1, random_state=0))]),
}

results = []
for name, pipe in models.items():
    stats = evaluate_model(name, pipe, X_train, y_train, X_test, y_test)
    results.append(stats)

results_df = pd.DataFrame(results).set_index('model')
results_df


### Ερμηνεία

1) `results_df` — Σύγκριση Linear / Ridge (α=1) / Lasso (α=0.1):
- Δες τις στήλες `MAE`, `RMSE` και `R2` για κάθε μοντέλο στο `results_df`.
- Σημείωση από τα τρέχοντα αποτελέσματα: οι τιμές είναι πολύ κοντινές μεταξύ των μοντέλων. Αυτό υποδεικνύει ότι για το συγκεκριμένο σύνολο χαρακτηριστικών και το split train/test, το απλό OLS (Linear) ήδη εξηγεί μεγάλο μέρος της διακύμανσης και η προσθήκη μικρής L2 ή L1 ποινής δεν αλλάζει πολύ το σφάλμα.
- Συγκεκριμένα, το RMSE του Ridge (α=1.0) είναι ελαφρώς μικρότερο από το RMSE του Linear — αυτό σημαίνει ότι μικρό shrinkage βοήθησε στη γενίκευση (μικρή μείωση σφάλματος στο test set). Η Lasso με α=0.1 συμπεριφέρεται σχεδόν όπως το Linear σε αυτές τις τιμές (πιθανότατα επειδή οι συντελεστές δεν ήταν αρκετά μεγάλοι ώστε να ‘κοπούν’ σε μηδενικά).


## 3. Επίδραση του `alpha` στα Ridge / Lasso

In [ ]:
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

ridge_rmse = []
lasso_rmse = []
ridge_coef_mean = []
lasso_coef_mean = []

for alpha in alphas:
    # Ridge
    ridge = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=alpha, random_state=0)),
    ])
    ridge.fit(X_train, y_train)
    y_pred_test_ridge = ridge.predict(X_test)
    rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_test_ridge))
    ridge_rmse.append(rmse_ridge)

    ridge_coef = ridge.named_steps['model'].coef_
    ridge_coef_mean.append(np.mean(np.abs(ridge_coef)))

    # Lasso
    lasso = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Lasso(alpha=alpha, random_state=0)),
    ])
    lasso.fit(X_train, y_train)
    y_pred_test_lasso = lasso.predict(X_test)
    rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_test_lasso))
    lasso_rmse.append(rmse_lasso)

    lasso_coef = lasso.named_steps['model'].coef_
    lasso_coef_mean.append(np.mean(np.abs(lasso_coef)))

summary_df = pd.DataFrame(
    {
        'alpha': alphas,
        'RMSE_Ridge': ridge_rmse,
        'RMSE_Lasso': lasso_rmse,
        'Mean|coef|_Ridge': ridge_coef_mean,
        'Mean|coef|_Lasso': lasso_coef_mean,
    }
)
summary_df


### Ερμηνεία

1) `summary_df` — Επίδραση του alpha στα Ridge / Lasso (πίνακας με γραμμές για κάθε alpha):
- Η στήλη `RMSE_Ridge` δείχνει πώς αλλάζει το RMSE του Ridge καθώς αυξάνουμε το α. Αν παρατηρήσουμε μια μικρή πτώση στο RMSE με μεγαλύτερα α (όπως στα τρέχοντα αποτελέσματα για α=100), αυτό σημαίνει ότι υψηλότερο regularization μείωσε λίγο την υπερεκπαίδευση χωρίς να εισάγει μεγάλο bias — δηλαδή βελτίωσε ελαφρώς την απόδοση στο test set.
- Η στήλη `Mean|coef|_Ridge` (μέσο απόλυτο μέγεθος των συντελεστών) δείχνει σαφώς shrinkage: όσο μεγαλώνει το α, τόσο μικραίνει το μέσο απόλυτο μέγεθος των συντελεστών. Αυτό είναι το αναμενόμενο αποτέλεσμα του L2: συρρίκνωση των β.
- Στη Lasso βλέπουμε λιγότερη αλλαγή στο RMSE στις δοκιμαζόμενες τιμές α και μικρότερη μείωση στο μέσο απόλυτο μέγεθος των συντελεστών · αυτό συμβαίνει επειδή για αυτές τις συγκεκριμένες τιμές α η L1 ποινή δεν είχε αρκετή ισχύ για να οδηγήσει πολλούς συντελεστές ακριβώς στο 0 (δηλαδή να δημιουργήσει αραιότητα).

2) Πρακτικά συμπεράσματα και οδηγίες:
- Αν το κύριο μέλημα είναι μικρή βελτίωση στην ακρίβεια πρόβλεψης (RMSE) στο test set, η Ridge με ένα μεσαίο-μεγάλο α μπορεί να βοηθήσει — δείτε πως το RMSE πέφτει ελαφρώς για μεγάλα α στην περίπτωσή μας.
- Εάν οι διαφορές μεταξύ Linear, Ridge και Lasso είναι πολύ μικρές (όπως εδώ), τότε προτεραιότητα μπορεί να δοθεί στην απλότητα (Linear) ή στην σταθερότητα (Ridge) ανάλογα με το επιθυμητό trade-off. 

